Quick one-off notebook to generate animations of value iteration at work.  Forked from the val_iter_GPU.ipynb notebook.



# Preliminaries

In [87]:


import numpy as np
import operator


import matplotlib.pyplot as plt


import IPython.display as dp
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec 
from mpl_toolkits.mplot3d import Axes3D

import time

import numba as nb

from timeit import default_timer as timer

In [95]:
!mkdir anim
def plot_frame( frame_num, V_o, final_V_o, min_V ):
    fig = plt.figure(figsize=(16,6), constrained_layout=False)
    gs = fig.add_gridspec(1,2)
    
    V_max = final_V_o.max()
    #V_min = final_V_o.min()
    V_min = min_V # supplied by caller

    ax = fig.add_subplot(gs[0, 0])
    plt.title( 'Value function, V (top view)')
    p = plt.imshow( V_o[1:-1,1:-1], cmap="coolwarm", vmax=V_max, vmin=V_min )
    plt.colorbar(p)

    ax = fig.add_subplot(gs[0, 1], projection='3d')
    plt.title( 'Value function, V (3D view)')

    X = np.arange(1, V_o.shape[0] - 1, 1.0)
    Y = np.arange(1, V_o.shape[1] - 1, 1.0)
    X, Y = np.meshgrid(X, Y)

    Z = V_o[1:-1,1:-1]

    #print( X.shape, V_o.shape )


    e=40
    a=30

    ax.view_init(elev=e, azim=a)
    p = ax.plot_surface(X, Y, Z, cmap='coolwarm',
                           linewidth=0, antialiased=False, vmax=V_max, vmin=V_min)
    plt.colorbar(p)
    ax.set_zlim3d( min_V, V_max )
    plt.savefig( 'anim/frame_%05d.png' % frame_num, format='png' )
    
    plt.close(fig)



mkdir: cannot create directory ‘anim’: File exists


In [124]:
#%load_ext autoreload
#%autoreload 2

#import os
#os.environ["NUMBA_ENABLE_CUDASIM"] = ""
#del os.environ["NUMBA_ENABLE_CUDASIM"]

import numpy as np
import math

import numba
from numba import cuda

@cuda.jit
def bellman_update(xmax, ymax, V_i, V_o, R_gpu, residual_gpu, gamma):
    
    pos = cuda.grid(1)

    #print( pos )
    if pos > xmax * ymax:
      return
  
    x = pos % ymax
    y = pos // ymax

    #print( "pos %d -> x %d, y %d" % (pos, x, y ) ) 
    if x >= xmax:
      return

    if y >= ymax:
      return

    # up
    xu = x 
    yu = max( 0, y - 1 )
    vu = V_i[ xu, yu ]

    # left
    xl = max( 0, x - 1 ) 
    yl = y
    vl = V_i[ xl, yl ]

    # down
    xd = x 
    yd = min( ymax-1, y + 1 )
    vd = V_i[ xd, yd ]

    # right
    xr = min( xmax-1, x + 1 ) 
    yr = y
    vr = V_i[ xr, yr ]

    # immediate reward
    r = R_gpu[ x, y ]

    # bellman update
    v = r + gamma * max( vu, vl, vd, vr )

    # residual 
    res = abs(v - V_i[ x, y ])

    # update the value 
    V_o[ x, y ] = v

    # atomically update the residual
    cuda.atomic.max( residual_gpu, 0, res ) 
    


def numba_value_iteration(dims, V, R, epsilon, gamma, plot=False, final_V_o=None, min_V=None, skip=1 ):
    # Move data to GPU so we can do two operations on it
    V_gpu_a = cuda.to_device(V)
    V_gpu_b = cuda.to_device(V)
    R_gpu = cuda.to_device(R)
    residual_log = []

    residual = np.array( [ -np.inf ], dtype=V.dtype )

    count = 0
    while count == 0 or ( residual[0] > epsilon ):   # and count < 100 ):

      # Set up a ping-pong between the two value function buffers
      # so that V_i (input) is fixed during the iteration and
      # V_o (output) is updated at each iteration.
      if count % 2 == 0:
        V_i = V_gpu_a
        V_o = V_gpu_b
      else:
        V_i = V_gpu_b
        V_o = V_gpu_a

      # reset residual for next run
      residual[0] = -np.inf
    
      residual_gpu = cuda.to_device( residual )

      num_threads = dims[0] * dims[1]
      threadsperblock = 256
      blockspergrid = math.ceil(num_threads / threadsperblock)

      ### Perform Bellman upate
      bellman_update[blockspergrid,threadsperblock](dims[0], dims[1], V_i, V_o, R_gpu, residual_gpu, gamma)

      residual = residual_gpu.copy_to_host()
      #print( "count %d: %f" % (count, residual[0]) )

      V_o_host = V_o.copy_to_host()
        
      if plot: 
          if skip is None or count % skip == 1:
              #np.save( 'val_iter/hist_%d.npy' % count, V_o_host )
              plot_frame( count, V_o_host, final_V_o, min_V )
      
      count += 1 
      residual_log.append( residual[0] )


    return V_o.copy_to_host(), residual_log



# Single positive reward

In [133]:


# The dimensions of the state space S is captured by dims.  
# Multiplying the dimensions together would yield the size of the
# state space |S|.
dim = 100
dims = (dim,dim)  # square grid world
size = np.prod( dims )

print( dims, size )

# The value function is the same size as the state space
V = np.zeros( dims )

# Our action space is up, left, down, right.  If we were to go beyond
# the boundaries of the state space by taking an action 'a', then 
# will will just stay in the current state 's'.
num_actions = 4
  

# For now, we are creating a reward function R( s ) which is only
# dependent on the state but not the action.
R = np.zeros( dims )

# define some rewards in the reward function
R[ 50,50 ] = 10


# define bellman residual threshold
epsilon = 1

# discount factor
gamma = 0.99

print( "Solving MDP" )

from timeit import default_timer as timer

# Solve first without plotting
start = timer()    
V_o, residual_log = numba_value_iteration( dims, V, R, epsilon, gamma )
end = timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282






(100, 100) 10000
Solving MDP
0.09275003802031279


In [134]:
print( 'Took %d iterations to reach Bellman residual of %f' % (len(residual_log), epsilon) )

Took 231 iterations to reach Bellman residual of 1.000000


In [137]:
# Now go back and plot now that we know the final answer, so we know maximum values and can color things appropriately
!rm anim/*.png

discard1, discard2 = numba_value_iteration( dims, V, R, epsilon, gamma, plot=True, final_V_o=V_o, min_V=-50, skip=None)


rm: cannot remove 'anim/*.png': No such file or directory


In [138]:
# Found ffmpeg works pretty well nowadays (6/14/22).  Note the pallete has to be generated first or the image quality
# will be poor.  This was generating gifs for me in maybe 15 seconds or so.

!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -vf palettegen palette.png
!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -i palette.png -lavfi paletteuse  single_positive.gif

Error: unable to open display 
ffmpeg version n4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix= --prefix=/usr --disable-debug --disable-doc --disable-static --enable-cuda --enable-cuda-sdk --enable-cuvid --enable-libdrm --enable-ffplay --enable-gnutls --enable-gpl --enable-libass --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libmp3lame --enable-libnpp --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopus --enable-libpulse --enable-sdl2 --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libv4l2 --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxvid --enable-nonfree --enable-nvenc --enable-omx --enable-openal --enable-opencl --enable-runtime-cpudetect --enable-shared --enable-vaapi --enable-vdpau --enable-version3 --enable-xlib
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.1

# Positive rewards

In [139]:


# The dimensions of the state space S is captured by dims.  
# Multiplying the dimensions together would yield the size of the
# state space |S|.
dim = 100
dims = (dim,dim)  # square grid world
size = np.prod( dims )

print( dims, size )

# The value function is the same size as the state space
V = np.zeros( dims )

# Our action space is up, left, down, right.  If we were to go beyond
# the boundaries of the state space by taking an action 'a', then 
# will will just stay in the current state 's'.
num_actions = 4
  

# For now, we are creating a reward function R( s ) which is only
# dependent on the state but not the action.
R = np.zeros( dims )

# define some rewards in the reward function
R[ 20,70 ] = 10
R[ 90,30 ] = 10

# define bellman residual threshold
epsilon = .05

# discount factor
gamma = 0.99

print( "Solving MDP" )

from timeit import default_timer as timer

# Solve first without plotting
start = timer()    
V_o, residual_log = numba_value_iteration( dims, V, R, epsilon, gamma )
end = timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282






(100, 100) 10000
Solving MDP
0.19718093052506447


In [140]:
print( 'Took %d iterations to reach Bellman residual of %f' % (len(residual_log), epsilon) )

Took 529 iterations to reach Bellman residual of 0.050000


In [141]:
# Now go back and plot now that we know the final answer, so we know maximum values and can color things appropriately
!rm anim/*.png

discard1, discard2 = numba_value_iteration( dims, V, R, epsilon, gamma, plot=True, final_V_o=V_o, min_V=0 )


## Convert image frames to animation

In [142]:
# Tried using imagemagick to convert to animated gif, but it is pretty slow.  Would not recommend.
# !convert -delay 20 anim/*.png -loop 0 anim.gif

In [143]:
# Found ffmpeg works pretty well nowadays (6/14/22).  Note the pallete has to be generated first or the image quality
# will be poor.  This was generating gifs for me in maybe 15 seconds or so.

!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -vf palettegen palette.png
!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -i palette.png -lavfi paletteuse  positive_rewards.gif

Error: unable to open display 
ffmpeg version n4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix= --prefix=/usr --disable-debug --disable-doc --disable-static --enable-cuda --enable-cuda-sdk --enable-cuvid --enable-libdrm --enable-ffplay --enable-gnutls --enable-gpl --enable-libass --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libmp3lame --enable-libnpp --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopus --enable-libpulse --enable-sdl2 --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libv4l2 --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxvid --enable-nonfree --enable-nvenc --enable-omx --enable-openal --enable-opencl --enable-runtime-cpudetect --enable-shared --enable-vaapi --enable-vdpau --enable-version3 --enable-xlib
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.1

# Positive and Negative rewards

In [144]:


# The dimensions of the state space S is captured by dims.  
# Multiplying the dimensions together would yield the size of the
# state space |S|.
dim = 100
dims = (dim,dim)  # square grid world
size = np.prod( dims )

print( dims, size )

# The value function is the same size as the state space
V = np.zeros( dims )

# Our action space is up, left, down, right.  If we were to go beyond
# the boundaries of the state space by taking an action 'a', then 
# will will just stay in the current state 's'.
num_actions = 4
  

# For now, we are creating a reward function R( s ) which is only
# dependent on the state but not the action.
R = np.zeros( dims )

# define some rewards in the reward function
R[ 20,70 ] = 10
R[ 90,30 ] = 10
R[ 80:90,80:90 ] = -50

# define bellman residual threshold
epsilon = .05

# discount factor
gamma = 0.99

print( "Solving MDP" )

from timeit import default_timer as timer

# Solve first without plotting
start = timer()    
V_o, residual_log = numba_value_iteration( dims, V, R, epsilon, gamma )
end = timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282






(100, 100) 10000
Solving MDP
0.1880950666964054


In [145]:
print( 'Took %d iterations to reach Bellman residual of %f' % (len(residual_log), epsilon) )
print( 'min V: %f, max V: %f' % (V_o.min(), V_o.max() ) )

Took 529 iterations to reach Bellman residual of 0.050000
min V: -1.340710, max V: 500.070257


In [146]:
# Now go back and plot now that we know the final answer, so we know maximum values and can color things appropriately
!rm anim/*.png

discard1, discard2 = numba_value_iteration( dims, V, R, epsilon, gamma, plot=True, final_V_o=V_o, min_V=-50, skip=None )


rm: cannot remove 'anim/*.png': No such file or directory


In [147]:
# Make the animation

!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -vf palettegen palette.png
!/snap/bin/ffmpeg -y -framerate 60 -pattern_type glob -i 'anim/*.png' -i palette.png -lavfi paletteuse  negative_rewards.gif


Error: unable to open display 
ffmpeg version n4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix= --prefix=/usr --disable-debug --disable-doc --disable-static --enable-cuda --enable-cuda-sdk --enable-cuvid --enable-libdrm --enable-ffplay --enable-gnutls --enable-gpl --enable-libass --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libmp3lame --enable-libnpp --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopus --enable-libpulse --enable-sdl2 --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libv4l2 --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxvid --enable-nonfree --enable-nvenc --enable-omx --enable-openal --enable-opencl --enable-runtime-cpudetect --enable-shared --enable-vaapi --enable-vdpau --enable-version3 --enable-xlib
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.1